In [1]:
import glob
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
%matplotlib inline
plt.style.use('ggplot')

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.titlesize'] = 13

In [28]:
class_labels = {
    'down' : 0,
    'go'   : 1,
    'left' : 2,
    'right': 3,
    'up'   : 4
}

def extract_feature(fn):
    
    X, _ = librosa.load(fn, sr=None)
    
    return np.pad(X, ((0),(16000 - len(X))), mode='constant', constant_values=0.0)
    

def parse_audio_files(parent_dir, sub_dirs, file_ext='*.wav'):
    
#     +32 -40 +360
    features, labels = np.empty((0, 16000)), []
    
    for label, sub_dir in enumerate(sub_dirs):
        
        raw_label_all_zero = np.zeros(5, dtype=np.int)
        print("Working on %s" % sub_dir)
        
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            
            audio_signal = extract_feature(fn)
            features = np.vstack([features, audio_signal])
            
            raw_label_all_zero[class_labels[sub_dir]] = 1
            labels.append(raw_label_all_zero)
            
    return np.array(features), np.array(labels, dtype = np.int)

In [39]:
parent_dir = 'dataset'

sub_dirs = ['down','go','left', 'right', 'up']

features, labels = parse_audio_files(parent_dir, sub_dirs)

Working on down
Working on go
Working on left
Working on right
Working on up


In [40]:
np.savetxt("nn_conv1d_features.csv", features, delimiter=",")
np.savetxt("nn_conv1d_labels.csv", labels, delimiter=",")

In [41]:
X_all = features
y_all = labels

In [42]:
from sklearn.model_selection import train_test_split

X_not_test, X_test, y_not_rest, y_test = train_test_split(
    X_all, y_all, stratify=labels, train_size=.85)

X_train, X_val, y_train, y_val = train_test_split(
    X_not_test, y_not_rest, stratify=y_not_rest, train_size=.95)

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(9549, 16000)
(9549, 5)
(503, 16000)
(503, 5)
(1774, 16000)
(1774, 5)


In [32]:
## Imports
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv1D
from keras import backend as K
from keras.utils import print_summary, plot_model

In [33]:
# Training parameters
batch_size = 32
num_classes = 5
epochs = 40
signal_length = 16000

X_train = X_train.reshape(X_train.shape[0], signal_length, 1)
X_test = X_test.reshape(X_test.shape[0], signal_length, 1)
X_val = X_val.reshape(X_val.shape[0], signal_length, 1)
input_shape = (signal_length, 1)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(25, 16000, 1)
(25, 5)


In [34]:
from keras import regularizers
from keras.layers import BatchNormalization

def build_model():
    # Model definition
    model = Sequential()
    model.add(Conv1D(10, kernel_size=int(16000 * 0.02),
                     strides=int(16000 * 0.01),
                     activation='relu',
                     input_shape=input_shape))
    
    model.add(Conv1D(20, kernel_size=10, 
                     strides=5,
                     activation='relu'))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))

    model.add(Dense(64, activation='relu'))
    
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(lr=.005),
                  metrics=['accuracy'])
    
    return model

In [35]:
model = build_model()

In [36]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))

Train on 25 samples, validate on 25 samples
Epoch 1/40
25/25 [==============================] - 1s 31ms/step - loss: 1.6077 - acc: 0.1200 - val_loss: 1.5736 - val_acc: 0.4800
Epoch 2/40
25/25 [==============================] - 0s 823us/step - loss: 1.5736 - acc: 0.4800 - val_loss: 1.5063 - val_acc: 0.6000
Epoch 3/40
25/25 [==============================] - 0s 1ms/step - loss: 1.5063 - acc: 0.6000 - val_loss: 1.4032 - val_acc: 0.5200
Epoch 4/40
25/25 [==============================] - 0s 1ms/step - loss: 1.4032 - acc: 0.5200 - val_loss: 1.2806 - val_acc: 0.5200
Epoch 5/40
25/25 [==============================] - 0s 2ms/step - loss: 1.2806 - acc: 0.5200 - val_loss: 1.1263 - val_acc: 0.6000
Epoch 6/40
25/25 [==============================] - 0s 1ms/step - loss: 1.1263 - acc: 0.6000 - val_loss: 0.9581 - val_acc: 0.7200
Epoch 7/40
25/25 [==============================] - 0s 1ms/step - loss: 0.9581 - acc: 0.7200 - val_loss: 0.7794 - val_acc: 0.7600
Epoch 8/40
25/25 [=========================

In [38]:
score = model.evaluate(X_test, y_test, verbose=1)
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

25/25 [==============================] - 0s 258us/step

Test loss: 2.00271685458e-07
Test accuracy: 1.0
